In [1]:
import sys

sys.path.append('/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/python')

In [2]:
import jax
from flax.linen import relu, softmax, sigmoid
from nn.jax.multi_layer_dense import MultiLayerDense

In [3]:
predict_fn = MultiLayerDense(features=[16, 32, 7], activations=[relu, relu, softmax ])

In [4]:
predict_fn_params = predict_fn.init(jax.random.PRNGKey(0), jax.random.uniform(jax.random.PRNGKey(0), (7,)))

In [5]:
def loss(parameters, embedding, label):
    prediction = predict_fn.apply(parameters, embedding)
    label_probs = prediction * label + (1 - prediction) * (1 - label)
    return -jax.numpy.sum(jax.numpy.log(label_probs))
def batch_loss(parameters, batch_embeddings, batch_labels):
    losses = jax.vmap(loss, (None, 0, 0))(parameters, batch_embeddings, batch_labels)
    return jax.numpy.sum(losses)

In [6]:
labels = jax.numpy.eye(7)

In [7]:
logits = jax.random.uniform(jax.random.PRNGKey(0), (7,7))

In [8]:
for i in range(100):
    

    primals, f_vjp = jax.vjp(batch_loss, predict_fn_params, logits, labels)

    print(primals)

    delta = jax.grad(batch_loss)(predict_fn_params, logits, labels)

    predict_fn_params = jax.tree_multimap(lambda x, y: jax.numpy.asarray(x - (y * 0.03)), predict_fn_params, delta)

20.200226
20.004143
19.852156
19.713505
19.5754
19.447178
19.309875
19.172783
19.038307
18.889912
18.721485
18.49134
18.27116
18.070759
17.828806
17.597996
17.32606
17.068993
16.785286
16.48259
16.145205
15.8378315
15.444893
15.105116
14.744544
14.343685
13.910082
13.536744
13.144804


KeyboardInterrupt: 

In [13]:
grad = jax.random.normal(jax.random.PRNGKey(0), (7,))

In [14]:
grad

DeviceArray([ 0.08086776, -0.3862472 , -0.37565574,  0.586919  ,
             -1.2758199 ,  2.1192    , -0.8582123 ], dtype=float32)

DeviceArray([[0.18625414, 0.11664049, 0.12677927, 0.20967744, 0.0867876 ,
              0.11547495, 0.15838608],
             [0.06403505, 0.3850401 , 0.07648731, 0.09188085, 0.09633212,
              0.15993685, 0.12628767],
             [0.08561444, 0.1012434 , 0.39764372, 0.05966002, 0.23319784,
              0.09354515, 0.02909541],
             [0.1226268 , 0.05804941, 0.02842175, 0.38419333, 0.02370915,
              0.08025877, 0.3027407 ],
             [0.06138471, 0.13136324, 0.30340755, 0.04641355, 0.3235324 ,
              0.10754378, 0.02635481],
             [0.09319254, 0.19130464, 0.09919877, 0.17527546, 0.10821319,
              0.18697563, 0.14583981],
             [0.09990012, 0.07945987, 0.0179309 , 0.23234688, 0.02038922,
              0.07706101, 0.47291195]], dtype=float32)

In [18]:
loss, fn = jax.vjp(predict_fn.apply, predict_fn_params, logits)

In [27]:
fn(jax.numpy.tile(grad,(7,1)))

(FrozenDict({
     params: {
         layers_0: {
             bias: DeviceArray([-0.417869  , -0.28870377,  0.03695109,  0.        ,
                          -0.8543307 , -0.39595792,  0.03321608,  0.7777498 ,
                           0.        , -0.05344198,  0.        , -0.67899156,
                           0.27177203,  0.3951215 ,  0.        , -0.1352368 ],            dtype=float32),
             kernel: DeviceArray([[-0.11205239,  0.09774604,  0.02027582,  0.        ,
                           -0.35716903, -0.29646242,  0.01877885,  0.09139813,
                            0.        , -0.04091439,  0.        , -0.21716794,
                            0.24913652,  0.15852769,  0.        , -0.01337179],
                          [-0.07546791,  0.17436166,  0.01589599,  0.        ,
                           -0.29320335, -0.31187725,  0.015913  , -0.02081963,
                            0.        , -0.04375859,  0.        , -0.1336625 ,
                            0.2591507 ,  0